In [33]:
import csv

from enbios2.const import BASE_DATA_PATH

base_path = BASE_DATA_PATH / "temp/miquel_upscaling"

# this is the sheet: "dendrogram-technology dict"
base_denodo_gram_file = base_path / "dendrogram_generation_base.csv"
# this is the PNIEC dendrogram sheet
dendogram_file = base_path / "dendrogram_generation_dendo.csv"
base_denodo_gram_file.exists(), dendogram_file.exists()

(True, True)

In [34]:
dendo_base_reader = csv.DictReader(base_denodo_gram_file.open(encoding="utf-8"))
print(dendo_base_reader.fieldnames)
all_base_rows = list(dendo_base_reader)

dendo_reader = csv.DictReader(dendogram_file.open(encoding="utf-8"))
print(dendo_reader.fieldnames)
dendo_rows = list(dendo_reader)

['Dendrogram name', 'tech name']
['Child', 'Parent', 'Dendrogram level']


In [35]:
# ok. lets match tech-name : Dendrogram name to "Child" (in dendo_rows)
base_dendo_names = set(row['Dendrogram name'] for row in all_base_rows)
base_dendo_names_in_dendo = set(
    row['Child'] for row in dendo_rows if row["Child"])  # the if will filter out the empty ones

In [36]:
# ok, that good. all names "Dendrogram name" appear somewhere as Child (of something)
base_dendo_names - base_dendo_names_in_dendo

set()

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from enbios2.experiment.sum_hierarchy import HierarchyNode

# ok lets build the hierarchy, or tree. dendogram is actually the last name I would give it :). Because that is the name of the plot
# we dont care about the "Dendrogram level" column, but just look at the Child - Parent relationship

base_tree: HierarchyNode = None
all_nodes: list[HierarchyNode] = []

for row in dendo_rows:
    if not row["Child"]:
        continue
    if not row["Parent"]:
        base_tree = HierarchyNode(row["Child"])
        all_nodes.append(base_tree)
    else:
        node = HierarchyNode(row["Child"].strip())
        parent_ = list(filter(lambda node: node.name == row["Parent"], all_nodes))
        if not parent_:
            print(row, "does not connect")
            print(all_nodes)
            continue
        parent = parent_[0]
        parent.add_child(node)
        all_nodes.append(node)

In [39]:
# ok, so far so good.
base_tree.assert_all_names_unique()
base_tree.as_dict()

{'electricity production': {'children': {'renewables': {'children': {'hydro': {'children': {},
      'value': None},
     'wind': {'children': {'onshore': {'children': {'<1MW': {'children': {},
          'value': None},
         '1-3MW': {'children': {}, 'value': None},
         '>3MW': {'children': {}, 'value': None}},
        'value': None},
       'offshore': {'children': {}, 'value': None}},
      'value': None},
     'photovoltaics': {'children': {'rooftop': {'children': {'monocrystalline': {'children': {},
          'value': None},
         'polycrystalline': {'children': {}, 'value': None}},
        'value': None},
       'openground': {'children': {}, 'value': None}},
      'value': None},
     'other renewables': {'children': {'biogas': {'children': {},
        'value': None},
       'biomass': {'children': {}, 'value': None},
       'solar thermoelectric': {'children': {}, 'value': None},
       'geothermal': {'children': {}, 'value': None}},
      'value': None}},
    'value

In [40]:
%%markdown
ok this was just a test if the basic tree can be build. now lets build a mega tree.
the root is called "impacts", which has 18 children, one for each indicator and each indicator has 2 children: onsite and offsite.
each of these 2 subtrees has some subtree of the kind we build before

ok this was just a test if the basic tree can be build. now lets build a mega tree.
the root is called "impacts", which has 18 children, one for each indicator and each indicator has 2 children: onsite and offsite.
each of these 2 subtrees has some subtree of the kind we build before


In [41]:
# however, the generator table include technology names, which are actually not in the tree, but the mapping that "dendrogram_generation_base.csv" is giving.
# so lets find all those "Dendrogram name" nodes and change their names to what the "tech name" column has...
for row in all_base_rows:
    #print(row)
    node = base_tree.find_child_by_name(row['Dendrogram name'])
    assert node
    node.name = row["tech name"]

In [42]:
generation_file = base_path / "PNIEC_generation.csv"
assert generation_file.exists()
generation_rows = list(csv.DictReader(generation_file.open(encoding="utf-8")))

# first lets check if "Technologies" match the "tech name"
base_tech_names = set(r["tech name"] for r in all_base_rows)
technologies = set(r["Technologies"] for r in generation_rows)

# el seems to be a good split token in the Technologies column
real_tech_names = set()

# lets at the same time build a map from Technologies -> tech name
technology2tree_tech_map: dict[str,str] = {}

for technology in technologies:
    assert "el" in technology
    parts = technology.split("_")
    el_index = parts.index("el")
    real_tech_name = "_".join(parts[:el_index])
    #print(real_tech_name)
    real_tech_names.add(real_tech_name)
    if real_tech_name in base_tech_names:
        technology2tree_tech_map[technology] = real_tech_name


(set(), set())

In [43]:
# ok lets build the 3 top layers with one iteration over the generation list collecting all the indicators
first_tech: str = None
indicators: list[str] = []

for row in generation_rows:
    if not first_tech:
        first_tech = row["Technologies"]
    if row["Technologies"] != first_tech:
        break
    indicators.append(row["Indicators"])
indicators

['terrestrial acidification potential (TAP)',
 'global warming potential (GWP1000)',
 'freshwater ecotoxicity potential (FETP)',
 'marine ecotoxicity potential (METP)',
 'terrestrial ecotoxicity potential (TETP)',
 'fossil fuel potential (FFP)',
 'freshwater eutrophication potential (FEP)',
 'marine eutrophication potential (MEP)',
 'human toxicity potential (HTPc)',
 'human toxicity potential (HTPnc)',
 'ionising radiation potential (IRP)',
 'agricultural land occupation (LOP)',
 'surplus ore potential (SOP)',
 'ozone depletion potential (ODPinfinite)',
 'particulate matter formation potential (PMFP)',
 'photochemical oxidant formation potential: humans (HOFP)',
 'photochemical oxidant formation potential: ecosystems (EOFP)',
 'water consumption potential (WCP)']

In [44]:
root = HierarchyNode("impacts")
indicator_subtrees = [] #  create this list to find it easier later
for indicator in indicators:
    indicator_subtree = HierarchyNode(indicator, [
        HierarchyNode("Onsite"), HierarchyNode("Offsite")
    ])
    root.add_child(indicator_subtree)
    indicator_subtrees.append(indicator_subtree)

root.as_dict()

{'impacts': {'children': {'terrestrial acidification potential (TAP)': {'children': {'Onsite': {'children': {},
      'value': None},
     'Offsite': {'children': {}, 'value': None}},
    'value': None},
   'global warming potential (GWP1000)': {'children': {'Onsite': {'children': {},
      'value': None},
     'Offsite': {'children': {}, 'value': None}},
    'value': None},
   'freshwater ecotoxicity potential (FETP)': {'children': {'Onsite': {'children': {},
      'value': None},
     'Offsite': {'children': {}, 'value': None}},
    'value': None},
   'marine ecotoxicity potential (METP)': {'children': {'Onsite': {'children': {},
      'value': None},
     'Offsite': {'children': {}, 'value': None}},
    'value': None},
   'terrestrial ecotoxicity potential (TETP)': {'children': {'Onsite': {'children': {},
      'value': None},
     'Offsite': {'children': {}, 'value': None}},
    'value': None},
   'fossil fuel potential (FFP)': {'children': {'Onsite': {'children': {},
      'value'

In [45]:
from copy import deepcopy

# each of the lowest nodes (onsite, offsite) now get their own copy of the main tree we created above
for indicator_sub_tree in indicator_subtrees:
    onsite, offsite = indicator_sub_tree.children
    onsite.join_tree(deepcopy(base_tree))
    offsite.join_tree(deepcopy(base_tree))

In [46]:
# lets do some introspection...

In [47]:
root # root has 18 children. all the indicator

[impacts - 18 children ]

In [48]:
#  first indicator, has 2 children, onsite, offsite
root.children[0], [kid for kid in root.children[0].children]

([terrestrial acidification potential (TAP) - 2 children (impacts)],
 [[Onsite - 2 children (terrestrial acidification potential (TAP))],
  [Offsite - 2 children (terrestrial acidification potential (TAP))]])

In [49]:
# going one step down, we see renewable and non-renewables, the base of our base tree
root.children[0].children[0], [kid.name for kid in root.children[0].children[0].children]

([Onsite - 2 children (terrestrial acidification potential (TAP))],
 ['renewables', 'non-renewables'])

In [51]:
# now we fill up all indicator sub-trees going row by row. finding the respective technology node for onsite and offsite and filling in the value
# we will use the technology2tree_tech_map, we created before to find the technology node in the tree
one_print = False
for index, row in enumerate(generation_rows):
    indicator_subtree: HierarchyNode = next(filter(lambda ind: row["Indicators"] == ind.name, indicator_subtrees))

    # use get, and check if it exits, in order to prevent
    tree_tech_name = technology2tree_tech_map.get(row["Technologies"], None)
    if not tree_tech_name:
        # print(row["Technologies"], "will be ignored")
        continue

    # the children are called Onsite and Offsite, exactly like the 2 columns with values
    for child in indicator_subtree.children:
        tech_node = child.find_child_by_name(tree_tech_name)
        if not tech_node:
            print("error", child.name, index, row["Technologies"], tree_tech_name)
            break
        tech_node.value = float(row[child.name]) #  child.name is either Onsite or Offsite...

    # assert tech_node

In [52]:
import json

json.dump(root.as_dict(), (base_path / "complete.json").open("w", encoding="utf-8"), indent=2)

In [53]:
root.calc()

382691932571.3334

In [54]:
json.dump(root.as_dict(), (base_path / "complete.json").open("w", encoding="utf-8"), indent=2)